In [1]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('../data/sentiment_tweets.csv',encoding = "ISO-8859-1")
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [2]:
#Data cleaning:
df.drop(['NO_QUERY', '1467810369'], inplace=True, axis=1)
df.columns = ['sentiment', 'date', 'nickname', 'tweet']
df

,sentiment,date,nickname,tweet
0,0,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
1,0,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
3,0,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,Mon Apr 06 22:20:00 PDT 2009,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...
1599994,4,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
# Noise removal, remove nickname from tweet
remove_noise = lambda tweet: re.sub('[^\w ]', '', re.sub(r'@\w+', '', tweet.lower().strip()))
df['tweet'] = df['tweet'].apply(remove_noise)
df['tweet']

0          is upset that he cant update his facebook by t...
1           i dived many times for the ball managed to sa...
2             my whole body feels itchy and like its on fire
3           no its not behaving at all im mad why am i he...
4                                         not the whole crew
                                 ...                        
1599994    just woke up having no school is the best feel...
1599995    thewdbcom  very cool to hear old walt intervie...
1599996    are you ready for your mojo makeover ask me fo...
1599997    happy 38th birthday to my boo of alll time tup...
1599998                              happy charitytuesday   
Name: tweet, Length: 1599999, dtype: object

In [4]:
#Tokenization and lemmatization
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess(tweet):
    tokens = word_tokenize(tweet)
    return " ".join(lemmatizer.lemmatize(token) for token in tokens if token not in stop_words)
df['tweet'] = df['tweet'].apply(preprocess)
df['tweet']

0          upset cant update facebook texting might cry r...
1          dived many time ball managed save 50 rest go b...
2                            whole body feel itchy like fire
3                                   behaving im mad cant see
4                                                 whole crew
                                 ...                        
1599994                        woke school best feeling ever
1599995    thewdbcom cool hear old walt interview â httpb...
1599996                       ready mojo makeover ask detail
1599997    happy 38th birthday boo alll time tupac amaru ...
1599998                                 happy charitytuesday
Name: tweet, Length: 1599999, dtype: object

In [5]:
#tf idf vectorize and train split
x_train, x_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2)
tfidf = TfidfVectorizer(ngram_range=(1, 3))
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

In [6]:
lr = LogisticRegression().fit(x_train, y_train)
lr.score(x_test, y_test)

/home/vladimir/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.792934375